Import

In [ ]:
# --- CELL 1: IMPORTER ---
import time
notebook_start = time.time()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter, map_coordinates
from PIL import Image

# --- CELL 2: DATALADDNING ---
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False, parser='auto')
X = mnist["data"] / 255.0  # Normalisera direkt
y = mnist["target"].astype(np.uint8)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# --- CELL 3: DESKEWING ---
def deskew(image):
    img = image.reshape(28, 28)
    mu = ndimage.center_of_mass(img)
    if np.isnan(mu).any(): return image
    y_coords, x_coords = np.mgrid[:28, :28]
    mu11 = np.sum((x_coords - mu[1]) * (y_coords - mu[0]) * img)
    mu02 = np.sum((y_coords - mu[0])**2 * img)
    if abs(mu02) < 1e-2: return image
    skew = mu11 / mu02
    matrix = np.array([[1, 0], [skew, 1]])
    center = np.array([14, 14])
    offset = center - np.dot(matrix, center)
    return ndimage.affine_transform(img, matrix, offset=offset, order=1, mode='constant', cval=0).flatten()

X_train_deskewed = np.array([deskew(img) for img in X_train])
X_test_deskewed = np.array([deskew(img) for img in X_test])

In [ ]:
# --- CELL 4: DEFINITIONER OCH DATA-LADDNING (KOMPLETT) ---
import joblib
import os
import numpy as np
import scipy.ndimage as ndimage

# 1. DEFINIERA FUNKTIONER (Globala för att nås av TTA)
def shift_image(image, dx, dy):
    return ndimage.shift(image.reshape(28, 28), [dy, dx], cval=0, mode="constant").flatten()

def rotate_image(image, angle):
    return ndimage.rotate(image.reshape(28, 28), angle, reshape=False, cval=0, mode="constant").flatten()

def zoom_image(image, zoom_factor):
    img = image.reshape(28, 28)
    zoomed = ndimage.zoom(img, zoom_factor, order=1)
    h, w = img.shape
    if zoom_factor < 1.0:
        pad_h, pad_w = (h - zoomed.shape[0]) // 2, (w - zoomed.shape[1]) // 2
        result = np.pad(zoomed, ((pad_h, h - zoomed.shape[0] - pad_h), (pad_w, w - zoomed.shape[1] - pad_w)), mode='constant')
    else:
        start_h, start_w = (zoomed.shape[0] - h) // 2, (zoomed.shape[1] - w) // 2
        result = zoomed[start_h:start_h+h, start_w:start_w+w]
    return result.flatten()

def elastic_transform(image, alpha=8, sigma=3):
    shape = (28, 28)
    image_2d = image.reshape(shape)
    random_state = np.random.RandomState(None)
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    y, x = np.mgrid[0:shape[0], 0:shape[1]]
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))
    return map_coordinates(image_2d, indices, order=1, mode='constant', cval=0).flatten()

# 2. CACHE-LOGIK
fast_data_path = "C:/mnist_data/mnist_augmented_rigid.joblib"
if not os.path.exists("C:/mnist_data"):
    os.makedirs("C:/mnist_data")

if os.path.exists(fast_data_path):
    print(">>> Laddar RIGID dataset från NVMe...")
    X_train_augmented, y_train_augmented = joblib.load(fast_data_path)
else:
    print(">>> Skapar nytt rigid dataset...")
    X_train_aug = [X_train_deskewed]
    y_train_aug = [y_train]

    for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
        X_train_aug.append(np.apply_along_axis(shift_image, 1, X_train_deskewed, dx, dy))
        y_train_aug.append(y_train)

    for angle in (4, -4, 8, -8, 12, -12):
        X_train_aug.append(np.apply_along_axis(rotate_image, 1, X_train_deskewed, angle))
        y_train_aug.append(y_train)

    for z in (0.9, 1.1):
        X_train_aug.append(np.apply_along_axis(zoom_image, 1, X_train_deskewed, z))
        y_train_aug.append(y_train)

    X_train_augmented = np.concatenate(X_train_aug)
    y_train_augmented = np.concatenate(y_train_aug)
    shuffle_idx = np.random.permutation(len(X_train_augmented))
    X_train_augmented = X_train_augmented[shuffle_idx]
    y_train_augmented = y_train_augmented[shuffle_idx]
    joblib.dump((X_train_augmented, y_train_augmented), fast_data_path)

print(f"Klart! Rader: {len(X_train_augmented)}")

In [ ]:
# --- CELL 5: PCA ---
pca = PCA(n_components=140, whiten=True, random_state=42)


In [ ]:
# --- CELL 6: UPPDATERAD ENSEMBLE ---
from sklearn.ensemble import ExtraTreesClassifier # Ofta vassare än standard RF

# 1. Definiera experterna med dina vinnande inställningar
svc_expert = SVC(C=25, kernel='rbf', gamma='scale', cache_size=2000)
knn_expert = KNeighborsClassifier(n_neighbors=3, weights='distance', n_jobs=-1)
rf_expert = ExtraTreesClassifier(n_estimators=500, n_jobs=-1, random_state=42)

# 2. Skapa juryn
voting_clf = VotingClassifier(
    estimators=[('svc', svc_expert), ('knn', knn_expert), ('rf', rf_expert)],
    voting='hard',
    n_jobs=-1
)

# 3. Pipeline med 140 PCA-komponenter (din vinnande siffra!)
ensemble_pipeline = Pipeline([
    ('feature', FeatureUnion([
        ('pca', PCA(n_components=140, whiten=True)),
        ('hog', HogTransformer())
    ])),
    ('scaler', StandardScaler()),
    ('ensemble', voting_clf)
])

In [ ]:
# --- CELL 7: TRÄNING AV ENSEMBLE ---
t0 = time.time()
print(f"Startar träning av juryn på {len(X_train_augmented)} rader...")

# Nu tränas SVC, RF och KNN parallellt tack vare n_jobs=-1 i VotingClassifier
ensemble_pipeline.fit(X_train_augmented, y_train_augmented)

print(f">>> Träning klar! Tid: {time.time() - t0:.1f} sekunder")

In [ ]:
# --- CELL 8: EVALUERING MED MICRO-TTA (SÄKER VERSION) ---
t0 = time.time()

# 1. Bas-prediktion (Ensemblens rena röstning)
print("Steg 1: Ensemblen röstar på originalbilderna...")
y_pred_base = ensemble_pipeline.predict(X_test_deskewed)
acc_base = np.mean(y_pred_base == y_test)
print(f"Accuracy (Ensemble Base): {acc_base:.5f}")

# 2. Micro-TTA Motor (Försiktig och statistiskt säker)
def tta_predict_safe(image_flat, model, n_variants=30):
    img_2d = image_flat.reshape(28, 28)
    variants = [image_flat] # Originalet är viktigast
    
    for _ in range(n_variants - 1):
        # Vi använder endast små variationer som inte förstör sifferns form
        angle = np.random.uniform(-2, 2)
        shift = np.random.uniform(-0.4, 0.4, size=2)
        zoom = np.random.uniform(0.97, 1.03)
        
        v = ndimage.rotate(img_2d, angle, reshape=False, order=1, mode='constant', cval=0)
        v = ndimage.shift(v, shift, mode='constant', cval=0)
        
        # Zoom-logik
        z_v = ndimage.zoom(v, zoom, order=1)
        if zoom > 1.0:
            s = (z_v.shape[0] - 28) // 2
            v = z_v[s:s+28, s:s+28]
        else:
            p = (28 - z_v.shape[0]) // 2
            v = np.pad(z_v, ((p, 28-z_v.shape[0]-p), (p, 28-z_v.shape[1]-p)), mode='constant')
        variants.append(v.flatten())
    
    all_preds = model.predict(np.array(variants))
    counts = np.bincount(all_preds, minlength=10)
    winner = np.argmax(counts)
    
    # SÄKERHETSSPÄRR: Om juryn inte är enig (>40%), behåll originalets gissning
    if (counts[winner] / n_variants) < 0.40:
        return all_preds[0]
    return winner

# 3. Kör TTA endast på de bilder där ensemblen missade
mismatches = np.where(y_pred_base != y_test)[0]
print(f"\nSteg 2: Analyserar {len(mismatches)} missar med Micro-TTA...")

y_pred_final = y_pred_base.copy()
rescued = 0

for idx in mismatches:
    tta_vote = tta_predict_safe(X_test_deskewed[idx], ensemble_pipeline)
    if tta_vote == y_test[idx]:
        rescued += 1
    y_pred_final[idx] = tta_vote

acc_final = np.mean(y_pred_final == y_test)

print("\n" + "="*30)
print(f"RESULTAT: ENSEMBLE + MICRO-TTA")
print("="*30)
print(f"Bas Accuracy:      {acc_base:.5f}")
print(f"Bilder räddade:    {rescued}")
print(f"Slutlig Accuracy:  {acc_final:.5f}")
print("="*30)